In [72]:
import pandas as pd
import numpy as np
import random
from scipy.stats import norm


def calculate_gas_price(temperature):
    base_price = 2.25  # Base price of gas
    temperature_effect = max(
        0, (temperature - 65) * 0.1
    )  # Adjust the coefficient as needed

    # Introduce randomness with a normal distribution
    randomness = np.round(norm.rvs(loc=0, scale=0.2), 2)  # Increase scale for more randomness

    # Combine temperature effect and randomness
    combined_effect = temperature_effect + randomness

    price_of_gas = base_price + combined_effect
    return np.round(price_of_gas, 2)


# Define the date range
start_date = "2020-01-01"
end_date = "2023-12-31"
dates = pd.date_range(start_date, end_date)

# Store locations
locations = ["Provo", "Springville", "Orem", "Lehi"]

# Holidays (fixed for simplicity)
holidays = pd.to_datetime(["2020-07-04", "2020-12-25", "2021-07-04", "2021-12-25"])

num_disaster_days = 15  # Increase the number of disaster days
natural_disaster_days = random.sample(
    pd.date_range(start="2020-01-01", end="2022-12-31", freq="D").tolist(),
    num_disaster_days,
)

# Generate data for each store for each day
records = []

for date in dates:
    weekday = date.weekday()
    quarter = (date.month - 1) // 3 + 1
    for location in locations:
        is_holiday = date in holidays
        is_day_after_holiday = (date - pd.Timedelta(days=1)) in holidays
        is_natural_disaster = date in natural_disaster_days
        is_day_after_disaster = (date - pd.Timedelta(days=1)) in natural_disaster_days

        # Simulate temperature with increased variability
        temperature = np.round(norm.rvs(loc=65, scale=30), 2)  # Increased scale for more randomness

        # Simulate gas price with increased variability
        price_of_gas = calculate_gas_price(temperature)

        # Simulate CPI with increased variability
        current_cpi = np.round(norm.rvs(loc=200, scale=25), 2)  # Increased scale for more randomness

        # Boost sales with more variability
        sales = np.round(norm.rvs(loc=1000, scale=500), 2)  # Increased standard deviation

        # Adjust sales based on holiday and disaster effects with increased variability
        if is_holiday:
            sales += np.round(norm.rvs(loc=600, scale=400), 2)  # Increased variability for holiday sales
        if is_natural_disaster:
            sales *= np.round(norm.rvs(loc=0.5, scale=0.15), 2)  # Adjust sales for natural disaster impact with more variability
        
        # Adjust sales for other factors with increased variability
        sales += np.round(norm.rvs(loc=100, scale=100), 2)  # Increased variability for other factors
        
        # Apply additional random noise to sales
        sales += np.random.normal(0, 200)  # Increased magnitude of noise

        # Increase sales on warmer days with increased variability
        sales = sales * (1 + 0.1 * (temperature - 65) / 65) + np.random.normal(0, 100) 

        # Increase sales on days with higher gas prices with increased variability
        sales += sales * 0.1 * (price_of_gas - 2.5) / 2.5 if price_of_gas > 2.5 else 0 + np.random.normal(0, 100)

        record = {
            "Date": date,
            "Weekday": weekday,
            "quarter": quarter,
            "StoreLocation": location,
            "Sales": max(0, sales),  # Ensure sales are non-negative
            "IsHoliday": is_holiday,
            "IsDayAfterHoliday": is_day_after_holiday,
            "IsDisaster": is_natural_disaster,
            "IsDayAfterDisaster": is_day_after_disaster,
            "Temperature": temperature,
            "PriceOfGas": price_of_gas,
            "CurrentCPI": current_cpi,
        }
        records.append(record)

df = pd.DataFrame(records)

# Process dataframe for regression (similar to your previous steps)
y = df["Sales"]
y = y.astype(float)
df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["day"] = df["Date"].dt.day
df["IsHoliday"] = df["IsHoliday"].astype(int)
df["IsDayAfterHoliday"] = df["IsDayAfterHoliday"].astype(int)
df["IsDisaster"] = df["IsDisaster"].astype(int)
df["IsDayAfterDisaster"] = df["IsDayAfterDisaster"].astype(int)

df.to_csv("../../outputs/sales_data.csv", index=False)

df = df.drop(columns=["Date"])
X = pd.get_dummies(df.drop(columns=["Sales"]), drop_first=True, dtype="int64")
X["IsHoliday"] = X["IsHoliday"].astype(int)
X["IsDayAfterHoliday"] = X["IsDayAfterHoliday"].astype(int)
X["IsDisaster"] = X["IsDisaster"].astype(int)
X["IsDayAfterDisaster"] = X["IsDayAfterDisaster"].astype(int)

df.head()

,Weekday,quarter,StoreLocation,Sales,IsHoliday,IsDayAfterHoliday,IsDisaster,IsDayAfterDisaster,Temperature,PriceOfGas,CurrentCPI,year,month,day
0,2,1,Provo,2182.692730,0,0,0,0,123.45,8.48,198.60,2020,1,1
1,2,1,Springville,915.414646,0,0,0,0,57.76,2.50,175.96,2020,1,1
2,2,1,Orem,294.359763,0,0,0,0,42.02,2.39,181.90,2020,1,1
3,2,1,Lehi,1199.379625,0,0,0,0,129.26,8.78,212.69,2020,1,1
4,3,1,Provo,1916.257951,0,0,0,0,46.59,2.19,144.88,2020,1,2


In [73]:
# df.to_csv("../../outputs/sales_data.csv", index=False)

In [74]:
import statsmodels.api as sm

model = sm.OLS(y, X).fit()


model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Sales   R-squared (uncentered):                   0.802
Model:                            OLS   Adj. R-squared (uncentered):              0.802
Method:                 Least Squares   F-statistic:                              1578.
Date:                Sat, 02 Mar 2024   Prob (F-statistic):                        0.00
Time:                        13:50:31   Log-Likelihood:                         -45437.
No. Observations:                5844   AIC:                                  9.090e+04
Df Residuals:                    5829   BIC:                                  9.100e+04
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Weekday                       2.8581      3.793      0.754      0.451      -4.578      10.294
quarter                      54.7640     28.471      1.924      0.054      -1.049     110.577
IsHoliday                   926.6226    144.757      6.401      0.000     642.846    1210.400
IsDayAfterHoliday           136.4898    144.648      0.944      0.345    -147.074     420.053
IsDisaster                 -452.8529     75.066     -6.033      0.000    -600.010    -305.695
IsDayAfterDisaster           77.1591     74.938      1.030      0.303     -69.747     224.065
Temperature                   0.8835      0.481      1.835      0.067      -0.060       1.827
PriceOfGas                   53.1860      8.313      6.398      0.000      36.889      69.483
CurrentCPI                   -0.1002      0.303     -0.330      0.741      -0.695       0.495
year                          0.4618      0.034     13.437      0.000       0.394       0.529
month                       -21.4103      9.221     -2.322      0.020     -39.487      -3.333
day                          -1.7766      0.858     -2.072      0.038      -3.458      -0.095
StoreLocation_Orem           23.0787     21.345      1.081      0.280     -18.766      64.923
StoreLocation_Provo          28.1863     21.342      1.321      0.187     -13.653      70.025
StoreLocation_Springville    43.1879     21.346      2.023      0.043       1.341      85.035
==============================================================================
Omnibus:                       18.833   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.749
Skew:                           0.128   Prob(JB):                     8.49e-05
Kurtosis:                       2.893   Cond. No.                     3.91e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict sales on the testing set
y_pred = model.predict(X_test)

# Convert the ordinal dates back to datetime for plotting
X_test["Date"] = pd.to_datetime(X_test["Date"].apply(datetime.datetime.fromordinal))

# Plotting
plt.figure(figsize=(10, 6))
plt.plot_date(X_test["Date"], y_test, "-", label="Actual Sales")
plt.plot_date(X_test["Date"], y_pred, "-", label="Predicted Sales")
plt.title("Sales Over Time")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.tight_layout()
plt.show()

KeyError: 'Date'